# Install Selenium

In [29]:
# Imports for selenium install
import os
import re
import subprocess
import requests

# The deb files we need to install
deb_files_startstwith = [
    "chromium-codecs-ffmpeg-extra_",
    "chromium-codecs-ffmpeg_",
    "chromium-browser_",
    "chromium-chromedriver_"
]

def get_latest_version() -> str:
    # A request to security.ubuntu.com for getting latest version of chromium-browser
    # e.g. "112.0.5615.49-0ubuntu0.18.04.1_amd64.deb"
    url = "http://security.ubuntu.com/ubuntu/pool/universe/c/chromium-browser/"
    r = requests.get(url)
    if r.status_code != 200:
        raise Exception("status_code code not 200!")
    text = r.text

    # Find latest version
    pattern = '<a\shref="chromium\-browser_([^"]+.ubuntu0\.18\.04\.1_amd64\.deb)'
    latest_version_search = re.search(pattern, text)
    if latest_version_search:
        latest_version = latest_version_search.group(1)
    else:
        raise Exception("Can not find latest version!")
    return latest_version

def download(latest_version: str, quiet: bool):
    deb_files = []
    for deb_file in deb_files_startstwith:
        deb_files.append(deb_file + latest_version)

    for deb_file in deb_files:
        url = f"http://security.ubuntu.com/ubuntu/pool/universe/c/chromium-browser/{deb_file}"

        # Download deb file
        if quiet:
            command = f"wget -q -O /content/{deb_file} {url}"
        else:
            command = f"wget -O /content/{deb_file} {url}"
        print(f"Downloading: {deb_file}")
        # os.system(command)
        !$command

        # Install deb file
        if quiet:
            command = f"apt-get install /content/{deb_file} >> apt.log"
        else:
            command = f"apt-get install /content/{deb_file}"
        print(f"Installing: {deb_file}\n")
        # os.system(command)
        !$command

        # Delete deb file from disk
        os.remove(f"/content/{deb_file}")

def check_chromium_installation():
    try:
        subprocess.call(["chromium-browser"])
        print("Chromium installation successfull.")
    except FileNotFoundError:
        print("Chromium Installation Failed!")

def install_selenium_package(quiet: bool):
    if quiet:
        !pip install selenium -qq >> pip.log
    else:
        !pip install selenium

def main(quiet: bool):
    # Get the latest version of chromium-browser for ubuntu 18.04
    latest_version = get_latest_version()
    # Download and install chromium-browser for ubuntu 20.04
    download(latest_version, quiet)
    # Check if installation succesfull
    check_chromium_installation()
    # Finally install selenium package
    install_selenium_package(quiet)

if __name__ == '__main__':
    quiet = True # verboseness of wget and apt
    main(quiet)

Downloading: chromium-codecs-ffmpeg-extra_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb
Installing: chromium-codecs-ffmpeg-extra_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb

Downloading: chromium-codecs-ffmpeg_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb
Installing: chromium-codecs-ffmpeg_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb

Downloading: chromium-browser_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb
Installing: chromium-browser_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb

Downloading: chromium-chromedriver_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb
Installing: chromium-chromedriver_112.0.5615.49-0ubuntu0.18.04.1_amd64.deb

Chromium installation successfull.


# Import Libraries

In [30]:
# Import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

# Other
import time
import csv
import random
from datetime import datetime
import pandas as pd

# Use Google Drive to save the data

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Scrape skillbridge

In [37]:
# Time delays
initial_wait_time = 15
table_load_wait_time = 5
random_delay_min = 3
random_delay_max = 8

# File Path
csv_file = "/content/drive/MyDrive/skillbridge/skillbridge.csv"
last_page = "/content/drive/MyDrive/skillbridge/last_page.txt"

In [48]:
# Setup webdriver
webdriver_options = Options()
webdriver_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36')
webdriver_options.add_argument('--headless')
webdriver_options.add_argument('--no-sandbox')
webdriver_options.add_argument('--disable-dev-shm-usage')
service = Service(executable_path=r'/usr/bin/chromedriver')
driver = webdriver.Chrome(service=service, options=webdriver_options)

# Navigate to the target webpage
driver.get("https://skillbridge.osd.mil/locations.htm")

#print(driver.page_source) # Debug

# Wait for the table data to load (adjust the timeout as needed)
time.sleep(10)  # Wait for 10 seconds
#driver.execute_script("window.scrollBy(0, 1000)")  # Scrolls down by 500 pixels
search_button = driver.find_element(By.ID, "loc-search-btn")
driver.execute_script("arguments[0].scrollIntoView(true);", search_button)
driver.execute_script("arguments[0].click();", search_button)

# Wait for the table to be present
time.sleep(table_load_wait_time)

#driver.save_screenshot("debug_screenshot.png") # Debug

# Open the CSV file
with open(csv_file, 'a', newline='') as file:
    writer = csv.writer(file)

    # Initialize page counter
    try:
        with open(last_page, 'r') as f:
            page_counter = int(f.read())
    except FileNotFoundError:
        page_counter = 1

    # If we're not on the first page, click the "Next" button to go to the desired page
    for _ in range(page_counter - 1):
        next_button = driver.find_element(By.ID, "location-table_next")
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(random.randint(random_delay_min, random_delay_max))

    # Loop to click the "Next" button and retrieve table data from each page
    while True:
        try:
            # If we're not on the first page, click the "Next" button to go to the next page
            if page_counter > 1:
                next_button = driver.find_element(By.ID, "location-table_next")
                driver.execute_script("arguments[0].click();", next_button)
                time.sleep(random.randint(random_delay_min, random_delay_max))

            # Refresh the table element to avoid stale element reference error
            table_element = driver.find_element(By.ID, "location-table")

            # Execute JavaScript to retrieve the innerText of all table cells
            cells_script = """
              var rows = document.querySelectorAll('#location-table tbody tr');
              var tableData = [];
              for (var i = 0; i < rows.length; i++) {
                  var rowData = [];
                  var cells = rows[i].querySelectorAll('td');
                  for (var j = 0; j < cells.length; j++) {
                      var cellText = cells[j].innerText.trim();
                      rowData.push(cellText);
                  }
                  tableData.push(rowData);
              }
              return tableData;
            """
            cell_texts = driver.execute_script(cells_script)
            print(f"Page {page_counter}: {len(cell_texts)} rows")
            # Add the cell texts to the tableData array and write to the CSV file
            for row in cell_texts:
                # Append page number and scrape date to the row data
                row_data = [cell.replace(',', ' ') for cell in row]
                row_data.append(page_counter)
                row_data.append(datetime.now().strftime('%Y-%m-%d'))
                writer.writerow(row_data)

            # Refresh the next button element to avoid stale element reference error | Check if the nextButton contains the "disabled" class
            next_button = driver.find_element(By.ID, "location-table_next")
            if "disabled" in next_button.get_attribute("class"):
                break  # Exit the loop

            # Increment page counter and save it to a file
            page_counter += 1
            with open(last_page, 'w') as f:
                f.write(str(page_counter))

        except Exception as e:
            # Handle any exceptions that occur during the process
            print(f"An error occurred: {str(e)}")
            break

# Clean up and quit the WebDriver instance
driver.quit()

Page 1: 14 rows
Page 2: 12 rows
Page 3: 14 rows
Page 4: 15 rows
Page 5: 11 rows
Page 6: 11 rows
Page 7: 11 rows
Page 8: 11 rows
Page 9: 14 rows
Page 10: 18 rows
Page 11: 13 rows
Page 12: 11 rows
Page 13: 13 rows
Page 14: 12 rows
Page 15: 14 rows
Page 16: 11 rows
Page 17: 11 rows
Page 18: 11 rows
Page 19: 11 rows
Page 20: 13 rows
Page 21: 11 rows
Page 22: 13 rows
Page 23: 14 rows
Page 24: 13 rows
Page 25: 11 rows
Page 26: 11 rows
Page 27: 11 rows
Page 28: 11 rows
Page 29: 11 rows
Page 30: 15 rows
Page 31: 11 rows
Page 32: 11 rows
Page 33: 11 rows
Page 34: 11 rows
Page 35: 11 rows
Page 36: 11 rows
Page 37: 13 rows
Page 38: 15 rows
Page 39: 14 rows
Page 40: 15 rows
Page 41: 18 rows
Page 42: 13 rows
Page 43: 18 rows
Page 44: 11 rows
Page 45: 12 rows
Page 46: 12 rows
Page 47: 11 rows
Page 48: 13 rows
Page 49: 14 rows
Page 50: 18 rows
Page 51: 14 rows
Page 52: 12 rows
Page 53: 15 rows
Page 54: 18 rows
Page 55: 16 rows
Page 56: 13 rows
Page 57: 12 rows
Page 58: 11 rows
Page 59: 11 rows
Page 6

In [49]:
# Open the input file in read mode and the output file in write mode
with open(csv_file, 'r') as input_file, open('csv_tmp.csv', 'w') as output_file:

    previous_row = ''
    for line in input_file:
        line = line.strip()  # Remove leading/trailing whitespace

        if line.startswith(','):  # Empty column found
            line = previous_row.split(',')[0] + line  # Fill with value from previous row

        if line.count(',') == 2:  # Ignore rows with exactly three columns
            previous_row = line
            continue

        output_file.write(line + '\n')  # Write the line to the output file
        previous_row = line

In [50]:
# Define column names
column_names = [
    "Company", "service", "city", "state", "duration_of_training",
    "employer_poc", "poc_email", "closest_installation",
    "location_of_job", "delivery_method", "target_moc",
    "prerequisites", "other", "job_description",
    "summary_description", "job_family", "mou_organization", "page_number", "scrape_date"
]

# Load the dataset with defined headers, skipping the first column
df = pd.read_csv("csv_tmp.csv", header=None, skipinitialspace=True, usecols=range(1, 20))

# Drop rows where all values are NaN
df = df.dropna(how='all')

# Assign the column names
df.columns = column_names

# Display the cleaned dataframe
df.head()

,Company,service,city,state,duration_of_training,employer_poc,poc_email,closest_installation,location_of_job,delivery_method,target_moc,prerequisites,other,job_description,summary_description,job_family,mou_organization,page_number,scrape_date
0,1 Acord Services LLC - Sunbum Solar,All Services,Virginia Beach,VA,151 - 180 days,Matt Shorrock,mshorrock@sunbumsolar.com,NaN,VA,In-person,All MOCs,NaN,All candidates are welcome to apply for this p...,Sunbum Solar is actively hiring to support the...,The Sunbum Solar training program provides an ...,Construction and Extraction,1 Acord Services LLC,1,2023-08-08
1,11th Contracting Squadron,Air Force Navy,Washington D.C.,DC,91 - 120 days,Terry Agena,terry.agena.1@us.af.mil,Joint Base Anacostia Bolling,DC,In-person,6C0X1,No prerequisite eligibility requirements,Members will work at 11th Contracting Squadron...,Federal Contracting Contract Administration ...,Members will work with 11th Contracting Squadr...,Other - Contracting,11th Contracting Squadron,1,2023-08-08
2,11th Judicial Circuit Veterans Treatment Court...,All Services,Bloomington,IL,151 - 180 days,Del Saam,Del.Saam@mcleancountyil.gov,NaN,IL,Hybrid (In-Person and Online),All,No prerequisite eligibility requirements,This is a job that actually makes a difference...,Future jobs could include probation officers ...,Veterans Treatment Court provides treatment se...,Community and Social Service; Healthcare Suppo...,11th Judicial Circuit Veterans Treatment Court...,1,2023-08-08
3,1PS Traders - Ecommerce Retail - PS Traders,All Services,Macomb,MI,151 - 180 days,Kent Hockenberry,kent@pstradersllc.com,Selfridge ANGB,Michigan,In-person,NaN,NaN,Spouses encouraged to apply as well,Position: Shipping & Receiving We sell mainly ...,We are a growing ecommerce business looking fo...,Sales and Related,NaN,1,2023-08-08
4,1PS Traders - Ecommerce Retail - PS Traders,All Services,Macomb,MI,151 - 180 days,Kent Hockenberry,kent@pstradersllc.com,NaN,Home Base is Macomb Michigan. We are open also...,In-person,NaN,Spouses encouraged to apply as well,We are looking for individuates who share our ...,We are a small business that has been growing ...,We are looking for a Shopper-Buyer to help us ...,Sales and Related,NaN,1,2023-08-08


In [51]:
# Define a function to compute the maximum duration of training
def compute_max_duration(duration_str):
    # Check if duration_str is not NaN
    if pd.notna(duration_str):
        # Extract numbers from the string
        numbers = list(map(int, re.findall(r'\d+', duration_str)))

        # Return the maximum of the numbers in months if the list is not empty, otherwise return NaN
        return max(numbers) / 30 if numbers else float('NaN')
    else:
        # Return NaN if duration_str is NaN
        return duration_str

# Apply the function to the 'duration_of_training' column
df['training_max_length_months'] = df['duration_of_training'].apply(compute_max_duration)

# Display the dataframe
df.head()

,Company,service,city,state,duration_of_training,employer_poc,poc_email,closest_installation,location_of_job,delivery_method,target_moc,prerequisites,other,job_description,summary_description,job_family,mou_organization,page_number,scrape_date,training_max_length_months
0,1 Acord Services LLC - Sunbum Solar,All Services,Virginia Beach,VA,151 - 180 days,Matt Shorrock,mshorrock@sunbumsolar.com,NaN,VA,In-person,All MOCs,NaN,All candidates are welcome to apply for this p...,Sunbum Solar is actively hiring to support the...,The Sunbum Solar training program provides an ...,Construction and Extraction,1 Acord Services LLC,1,2023-08-08,6.0
1,11th Contracting Squadron,Air Force Navy,Washington D.C.,DC,91 - 120 days,Terry Agena,terry.agena.1@us.af.mil,Joint Base Anacostia Bolling,DC,In-person,6C0X1,No prerequisite eligibility requirements,Members will work at 11th Contracting Squadron...,Federal Contracting Contract Administration ...,Members will work with 11th Contracting Squadr...,Other - Contracting,11th Contracting Squadron,1,2023-08-08,4.0
2,11th Judicial Circuit Veterans Treatment Court...,All Services,Bloomington,IL,151 - 180 days,Del Saam,Del.Saam@mcleancountyil.gov,NaN,IL,Hybrid (In-Person and Online),All,No prerequisite eligibility requirements,This is a job that actually makes a difference...,Future jobs could include probation officers ...,Veterans Treatment Court provides treatment se...,Community and Social Service; Healthcare Suppo...,11th Judicial Circuit Veterans Treatment Court...,1,2023-08-08,6.0
3,1PS Traders - Ecommerce Retail - PS Traders,All Services,Macomb,MI,151 - 180 days,Kent Hockenberry,kent@pstradersllc.com,Selfridge ANGB,Michigan,In-person,NaN,NaN,Spouses encouraged to apply as well,Position: Shipping & Receiving We sell mainly ...,We are a growing ecommerce business looking fo...,Sales and Related,NaN,1,2023-08-08,6.0
4,1PS Traders - Ecommerce Retail - PS Traders,All Services,Macomb,MI,151 - 180 days,Kent Hockenberry,kent@pstradersllc.com,NaN,Home Base is Macomb Michigan. We are open also...,In-person,NaN,Spouses encouraged to apply as well,We are looking for individuates who share our ...,We are a small business that has been growing ...,We are looking for a Shopper-Buyer to help us ...,Sales and Related,NaN,1,2023-08-08,6.0


In [52]:
df.to_csv('skillbridge_cleaned_processed.csv', index=False)

# Search for remote jobs that are cyber focused

In [53]:
df = pd.read_csv('skillbridge_cleaned_processed.csv')

# List of computer-related terms to look for in the 'job_family' field
computer_related_terms = ['Cyber', 'Cybersecurity', 'Software Development', 'Programming',
                          'Analyst', 'Programmer' ]

# Build the regular expression pattern
pattern = '|'.join(computer_related_terms)

# Filter the dataframe based on the conditions
remote_cyber_jobs_df = df[(df['delivery_method'].str.contains('Hybrid|Online', case=False, na=False)) &
                          (df['job_family'].str.contains(pattern, case=False, na=False))]

# Select the necessary columns
output_remote_cyber_jobs_df = remote_cyber_jobs_df[['Company', 'job_description', 'job_family', 'duration_of_training', 'delivery_method']]


# Save the output dataframe to a CSV file
output_remote_cyber_jobs_df.to_csv('cyber_jobs.csv', index=False)
output_remote_cyber_jobs_df.head()

,Company,job_description,job_family,duration_of_training,delivery_method
34,7 Eagle Group,There are a multitude of job titles but here a...,Other - Cybersecurity and Information Technology,91 - 120 days,Hybrid (In-Person and Online)
35,7 Eagle Group,There are a multitude of job titles but here a...,Other - Cybersecurity and Information Technology,91 - 120 days,Hybrid (In-Person and Online)
36,7 Eagle Group,There are a multitude of job titles but here a...,Other - Cybersecurity and Information Technology,91 - 120 days,Hybrid (In-Person and Online)
37,7 Eagle Group,There are a multitude of job titles but here a...,Other - Cybersecurity and Information Technology,91 - 120 days,Hybrid (In-Person and Online)
38,7 Eagle Group,There are a multitude of job titles but here a...,Other - Cybersecurity and Information Technology,91 - 120 days,Hybrid (In-Person and Online)


# Jobs that require a TS Security Clerance

In [54]:
df = pd.read_csv('skillbridge_cleaned_processed.csv')

# Filter the dataframe based on the condition
ts_jobs_df = df[df['prerequisites'].str.contains('Top Secret|Clerance', case=False, na=False)]

# Select the necessary columns
output_ts_jobs_df = ts_jobs_df[['Company', 'prerequisites','job_description', 'job_family', 'duration_of_training', 'delivery_method']]

output_ts_jobs_df.to_csv('ts_jobs.csv', index=False)
output_ts_jobs_df.head()

,Company,prerequisites,job_description,job_family,duration_of_training,delivery_method
549,Ascendant Program Services LLC,Active Top Secret Clearance & Minimum 4 years ...,Construction Management & Craft Positions in S...,Construction and Extraction,151 - 180 days,In-person
738,BlueHalo,Applicants should have familiarity with any of...,Software Engineer Software Developer Programmer,Computer and Mathematical,91 - 120 days,Hybrid (In-Person and Online)
739,BlueHalo,A skill assessment will be conducted prior to ...,Analyst Network Analyst Data Analyst Data S...,Military Specific and Computer and Mathematical,91 - 120 days,Hybrid (In-Person and Online)
1496,Department of Homeland Security (DHS) - Nation...,Experience using a version of ESRI's ArcMap A...,GIS Production Watch or Duty Officer in an Eme...,Other - Civil Service Emergency Management,151 - 180 days,In-person
1497,Department of Homeland Security (DHS) - Nation...,Must have an active and verifiable DOD Secret ...,Watch or Duty Officer in an Emergency Operatio...,Other - Civil Service Emergency Management,151 - 180 days,In-person
